In [1]:
from kafka import KafkaConsumer
import json
import csv
import os
import time
from datetime import datetime

In [6]:
# ==========================================
# ⚙️ CONFIGURATION
# ==========================================
KAFKA_TOPIC = 'traffy_data'
KAFKA_BROKER = 'localhost:9092'
GROUP_ID = 'traffy_raw_writer_group' 
OUTPUT_FILE = '../../data/raw/bangkok_traffy_raw.csv'
FLUSH_INTERVAL = 100 
TIMEOUT_MS = 5000 

# Header: เอาตาม Field จริงที่ Traffy ส่งมา + processing_time
CSV_FIELDS = [
    "ticket_id", "type", "organization", "comment", "photo", 
    "photo_after", "coords", "address", "subdistrict", "district", 
    "province", "timestamp", "state", "star", "count_reopen", 
    "last_activity", "processing_time"
]

# ==========================================
# 1. SETUP CONSUMER & FILE
# ==========================================
consumer = KafkaConsumer(
    KAFKA_TOPIC,
    bootstrap_servers=[KAFKA_BROKER],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id=GROUP_ID,
    value_deserializer=lambda x: json.loads(x.decode('utf-8')) if x else None,
    consumer_timeout_ms=TIMEOUT_MS
)

In [7]:
print(f"🚀 เริ่มบันทึกข้อมูลดิบ (Raw Data) ลงไฟล์: {OUTPUT_FILE}")

# สร้างโฟลเดอร์ถ้ายังไม่มี
folder_path = os.path.dirname(OUTPUT_FILE)
if folder_path and not os.path.exists(folder_path):
    os.makedirs(folder_path)

# เปิดไฟล์ CSV เพื่อเขียน
csv_file = open(OUTPUT_FILE, 'w', newline='', encoding='utf-8-sig') 
writer = csv.DictWriter(csv_file, fieldnames=CSV_FIELDS)
writer.writeheader()

# 🔥 สร้าง Set ไว้จำ ID ที่เคยเขียนลงไฟล์แล้ว
seen_ticket_ids = set()

print(f"🚀 เริ่มบันทึก...")

# ==========================================
# 2. MAIN LOOP
# ==========================================
try:
    count = 0
    duplicate_count = 0
    missing_id_count = 0
    
    for message in consumer:
        raw_data = message.value
        if raw_data is None: continue

        ticket_id = raw_data.get('ticket_id')


        if ticket_id in seen_ticket_ids:
            duplicate_count += 1
            continue

        if ticket_id is None:
            missing_id_count += 1
            continue
        
        # ✅ ถ้ายังไม่เคยเจอ:
        # 1. จดจำ ID นี้ไว้
        seen_ticket_ids.add(ticket_id)

        # 2. บันทึกลงไฟล์
        try:
            raw_record = {
                "ticket_id": ticket_id,
                "type": raw_data.get('type'),            
                "organization": raw_data.get('organization'),
                "comment": raw_data.get('comment'),
                "photo": raw_data.get('photo'),
                "photo_after": raw_data.get('photo_after'),
                "coords": raw_data.get('coords'),        
                "address": raw_data.get('address'),
                "subdistrict": raw_data.get('subdistrict'),
                "district": raw_data.get('district'),
                "province": raw_data.get('province'),
                "timestamp": raw_data.get('timestamp'),
                "state": raw_data.get('state'),
                "star": raw_data.get('star'),          
                "count_reopen": raw_data.get('count_reopen'),
                "last_activity": raw_data.get('last_activity'),
                "processing_time": datetime.now().isoformat() 
            }

            # เช็ค coords
            coords = raw_record["coords"]
            if coords is None:
                print(f"\n⚠️ ข้ามรายการที่ไม่มีพิกัด coords (ticket_id: {ticket_id})")
                continue

            writer.writerow(raw_record)
            count += 1
            
            if count % FLUSH_INTERVAL == 0:
                csv_file.flush()
                print(f"✅ Saved {count} (Duplicate: {duplicate_count}) (Missing: {missing_id_count})", end='\r')
            
        except Exception as e:
            print(f"\n⚠️ Error: {e}")
    
    print(f"\n🎉 ข้อมูลหมดแล้ว! (Timeout {TIMEOUT_MS}ms)")

except KeyboardInterrupt:
    print(f"\n🛑 User Stopped.")

finally:
    csv_file.flush()
    csv_file.close()
    consumer.close()
    
    print("-" * 40)
    print(f"📊 สรุปผลการทำงาน:")
    print(f"   - บันทึกจริง: {count} รายการ")
    print(f"   - ข้ามที่ซ้ำ: {duplicate_count} รายการ")
    print(f"📂 ไฟล์อยู่ที่: {OUTPUT_FILE}")

🚀 เริ่มบันทึกข้อมูลดิบ (Raw Data) ลงไฟล์: ../../data/raw/bangkok_traffy_raw.csv
🚀 เริ่มบันทึก...
✅ Saved 778200 (Duplicate: 8771) (Missing: 0)
🎉 ข้อมูลหมดแล้ว! (Timeout 5000ms)
----------------------------------------
📊 สรุปผลการทำงาน:
   - บันทึกจริง: 778255 รายการ
   - ข้ามที่ซ้ำ: 8771 รายการ
📂 ไฟล์อยู่ที่: ../../data/raw/bangkok_traffy_raw.csv
